In [1]:
%run StdPackages.ipynb

No clean-up of work-folder


# nestingTree.py

Nesting tree class takes data on nesting and merge them into one aggregate nesting tree. It then produces a number of mappings/subsets that can be useful, in particular for the CGE model generator. The file includes two different classes of nesting trees (individual and aggregate trees), as well as two "children" classes that are basically the same, but let you initialize from a specified data file. The following goes through the individual trees first, and then the aggregate class.


### Initialize

Initializing the tree we specify:
* ```name```: Only mandatory argument; name of the tree.
* ```self.tree:``` List of tuples defining the nesting structure. The tuples should include three elements with (1) sector index, (2) knot, (3) branch.
* ```self.io:``` Specifies whether the type of nest is an input type ('inp') or an output type ('out'). The default is input types, where the branches are "below" the knots in the tree. The output type reverses this; this is used e.g. for sectors that produce multiple outputs.
* ```self.f:``` Specifies the type of function to apply in the specific nest (used for CGE generator). If ```self.io == 'inp'``` the default behavior is 'CES', if ```self.out == 'out'``` default is 'CET'.
* ```self.scalePreserving:``` Specifies whether or not the nest is ```scale-preserving```, i.e. has the characteristic that the sum of branches equals the knot. This is automatically checked for some functions, defined by the tuple ```_scalePreserving```. This currently recognizes ```self.f``` belonging to 'CES_norm','CET_norm','MNL','MNL_out'; if other scale-preserving functions are used, the ```self.scalePreserving``` has to adjusted manually.
* ```self.ns:``` A dictionary that serves as a namespace. This is used to write code that always refer to the same name (keys in self.ns), with symbols that have names that are adjusted to the specific tree. Adjustments to this default namespace is done using ```**kwargs``` when initializing.
* ```self.db:``` a dictionary of ```gpy``` symbols that are stored for later use (see the section on trees' attributes).

Here we give an example of a nesting tree with two different sectors, and a mix of input/output trees:

In [2]:
t1 = nestingTree.tree('t1', tree = [('s1','Y','L'),('s1','Y','KE'),('s1','KE','x1'),('s1','KE','x2')])
t2 = nestingTree.tree('t2', tree = [('s2','Y','X'),('s2','Y','K')], **{'map': 'NewMapName'})  # this gives the attribute 'map' the name 'NewMapName'
t3 = nestingTree.tree('t3', tree = [('s2','Y1','Y'),('s2','Y2','Y')], io = 'output')

### Individual trees' attributes

Beyond the attributes that are created at initialization, the main attributes are defined from running the method ```self.attrs_from_tree()```. This generates a number of symbols and add them to the database in ```self.db```. These can be accessed by slicing ```self[x]``` with 'x' using the static definitions from the keys of the namespace. To get the pandas representation (slicing returns ```gpy``` symbol), use the method ```self.get``` instead.

In [3]:
[t.attrs_from_tree() for t in (t1,t2,t3)];

```self.map:``` The nesting structure as a pandas multiindex

In [4]:
print(f"Name of symbol: '{t1.ns['map']}'\n",
      f"Symbol: \n{t1.get('map')}")

Name of symbol: 'map_t1'
 Symbol: 
MultiIndex([('s1',  'Y',  'L'),
            ('s1',  'Y', 'KE'),
            ('s1', 'KE', 'x1'),
            ('s1', 'KE', 'x2')],
           names=['s', 'n', 'nn'])


```self.knot:``` Subset of knots in the tree

In [5]:
print(f"Name of symbol: '{t1.ns['knot']}'\n",
      f"Symbol: \n{t1.get('knot')}")

Name of symbol: 'knot_t1'
 Symbol: 
MultiIndex([('s1',  'Y'),
            ('s1', 'KE')],
           names=['s', 'n'])


```self.branch:``` Subset of branches in the tree

In [6]:
print(f"Name of symbol: '{t1.ns['branch']}'\n",
      f"Symbol: \n{t1.get('branch')}")

Name of symbol: 'branch_t1'
 Symbol: 
MultiIndex([('s1',  'L'),
            ('s1', 'KE'),
            ('s1', 'x1'),
            ('s1', 'x2')],
           names=['s', 'n'])


```self.n:``` All goods (inputs, intermediates, final goods)

In [7]:
print(f"Name of symbol: '{t1.ns['n']}'\n",
      f"Symbol: \n{t1.get('n')}")

Name of symbol: 'n'
 Symbol: 
Index(['KE', 'L', 'Y', 'x1', 'x2'], dtype='object', name='n')


```self.s:``` Set of sectors.

In [8]:
print(f"Name of symbol: '{t1.ns['s']}'\n",
      f"Symbol: \n{t1.get('s')}")

Name of symbol: 's'
 Symbol: 
Index(['s1'], dtype='object', name='s')


```self.input:``` Inputs in the tree.

In [9]:
print(f"Name of symbol: '{t1.ns['input']}'\n",
      f"Symbol: \n{t1.get('input')}")

Name of symbol: 'input_t1'
 Symbol: 
MultiIndex([('s1',  'L'),
            ('s1', 'x1'),
            ('s1', 'x2')],
           names=['s', 'n'])


```self.output:``` Outputs from the tree.

In [10]:
print(f"Name of symbol: '{t1.ns['output']}'\n",
      f"Symbol: \n{t1.get('output')}")

Name of symbol: 'output_t1'
 Symbol: 
MultiIndex([('s1', 'Y')],
           names=['s', 'n'])


## ```aggTree``` class

The aggregate tree takes a name, a dictionary of trees, and a namespace (all optional):

In [11]:
T = nestingTree.aggTree(name='agg', trees ={ti.name: ti for ti in (t1,t2,t3)})

The aggregate tree works in somewhat the same way as the individual ones: There is a namespace implemented that allows to write code that refers to static names. Symbols are now stored in a ```GpyDB``` accessed as ```self.db```. Beyond its ows namespace, we can also access the individual trees that are added from the ```self.trees```. We navigate this using the ```self.n,self.get``` methods: ```self.n``` accesses the name of the symbol, ```self.get``` returns the pandas-like representation.

The main method is the ```self__call__(self,namespace=None)```. This defines global symbols and adjusts trees. The namespace option allows for trees to use temporary names for set elements, e.g. if a nesting tree includes the same name for final inputs and outputs:

In [12]:
T()

*Access global variable:*

In [13]:
print(f"Name of symbol, global scope: \n'{T.n('map')}'\n",
      f"Symbols: \n{T.get('map')}")

Name of symbol, global scope: 
'map_agg'
 Symbols: 
MultiIndex([('s1',  'Y',  'L'),
            ('s1',  'Y', 'KE'),
            ('s1', 'KE', 'x1'),
            ('s1', 'KE', 'x2'),
            ('s2',  'Y',  'X'),
            ('s2',  'Y',  'K'),
            ('s2', 'Y1',  'Y'),
            ('s2', 'Y2',  'Y')],
           names=['s', 'n', 'nn'])


*Access local variable (in individual tree):*

In [14]:
print(f"Name of symbol, local scope: \n'{T.n('map',local='t1')}'\n",
      f"Symbols: \n{T.get('map',local='t1')}")

Name of symbol, local scope: 
'map_t1'
 Symbols: 
MultiIndex([('s1',  'Y',  'L'),
            ('s1',  'Y', 'KE'),
            ('s1', 'KE', 'x1'),
            ('s1', 'KE', 'x2')],
           names=['s', 'n', 'nn'])


The aggregate tree collects individual trees and aggregates them into a single nesting structure. The methods and standard attributes tailored to the methods in ```CGE_Generator``` classes, to help identify exogenous/endogenous subsets and efficiently write gams code.

### Relevant symbols:
In the aggregate tree:
* ```n```: Goods.
* ```s```: sectors.
* ```map```: entire nesting tree.
* ```input```: Inputs in aggregate tree.
* ```output```: Outputs from aggregate tree.
* ```int```: Intermediate goods.
* ```map_spinp```: Subset of ```map``` that are characterized by the individual tree being input-type and scale-preserving (```tree.scalePreserving = True, tree.io = 'inp'```).
* ```map_spout```: Subset of ```map``` that are characterized by the individual tree being output-type and scale-preserving (```tree.scalePreserving = True, tree.io = 'out'```).
* ```knout```: Knots in the nesting tree from output-type trees (```tree.io = 'out'```).
* ```kninp```: Knots in the nesting tree from input-type trees (```tree.io = 'inp'```).
* ```spinp```: Subset of knots in tree that are both input type + scale preserving.
* ```spout```: Subset of knots in tree that are both output type + scale preserving.

Symbols added that are specific to individual tree:
* **For input type trees:**
    * ```knot_o```:  The subset of knots that are outputs from the aggregate tree. 
    * ```knot_no```: The subset of knots thare are not outputs from aggregate tree.
    * ```branch2o```: Branches for which the relevant knot is an output from aggregate tree.
    * ```branch2no```: Branches for which the relevant knot is not an output from aggregate tree.
* **For output type trees:**
    * ```branch_o```: Branches that are outputs from aggregate tree.
    * ```branch_no```: Branches that are not outputs from aggregate tree.

#### Some examples:

*No knots in the tree ```t2``` are outputs, all are branches that are tied to non-outputs:*

In [15]:
T.get('knot_o',local='t2')

MultiIndex([], names=['s', 'n'])

In [16]:
T.get('branch2no',local='t2')

MultiIndex([('s2', 'X'),
            ('s2', 'K')],
           names=['s', 'n'])

*For the tree t3 all branches are outputs:*

In [17]:
T.get('branch_o',local='t3')

MultiIndex([('s2', 'Y1'),
            ('s2', 'Y2')],
           names=['s', 'n'])

## ```aggTree_from_data```

The class initializes the tree from excel data. The following tests reading in and processing various nesting trees.

This goes through identical steps for:
* CES1: A simple CES sector.
* CES2: A nested CES sector (2 nests).
* CES1_norm: Normalized CES sector.
* CES2_norm: Nested and normalized CES sector.
* CET1: A simple CET sector.
* CET2: A nested CET sector. 
* CET1_norm: A normalized CET sector.
* CET2_norm: A nested, normalized CET sector.
* CES_CET: A nested CES, CET sector (CES to a intermediate good, that is split into two outputs CET).
* CES_CET_norm: A nested CES, CET normalized sector (same as CES_CET, but CET sector has normalized technology).
* CESCET_norm: A nested, normalized CES,CET sector (same as CES_CET, but both sectors have normalized tech).
* FunkyTree: Mixes normalized input and output trees: Z1 is an input that is split into an output Y1 and an intermediate good X1. Similarly, Z2 is split into output Y2 and intermediate good X2. Finally, X1 and X2 are combined to a single output Y3. 
* ExtremeFunk: Mixes normalized input and output trees to highlight the role of choosing the right $\mu$s to keep exogenous when calibrating.
* FunnyName: Is identical to the ExtremeFunk tree, but tests the ```namespace``` feature when specifying trees. This feature is used when a sector produces one good (say ```Y```), but it also consumes this good as well. Essentially, we have two nodes in the tree that is called ```Y```, but they need to be distinct nodes. We do this by temporarily giving another name to one of them (e.g. ```Y_input```), and then, when reading the nesting tree, we specify a namespace ```namespace = dict(Y_input=Y)```. The two nodes (```Y_input, Y```) are processed as two separate nodes, but, after establishing links of the tree, the node ```Y_input``` is renamed ```Y``` again. 

*Init nesting structures:*

In [18]:
FunnyName_NS = {n+'_input': n for n in ('Y1','Y2')}
data_str = os.path.join(d['data'], 'Nestings.xlsx')
read_trees = {'CES1': {'CES1': {'f': 'CES'}},
              'CES2': {'CES2': {'f':'CES'}},
              'CES1_norm': {'CES1': {'f':'CES_norm'}}, 
              'CES2_norm': {'CES2': {'f':'CES_norm'}},
              'CET1': {'CET1': {'f': 'CET'}},
              'CET2': {'CET2': {'f': 'CET'}},
              'CET1_norm': {'CET1': {'f': 'CET_norm'}},
              'CET2_norm': {'CET2': {'f': 'CET_norm'}},
              'CES_CET': {'CES1': {'f': 'CES'}, 'CET1': {'f': 'CET'}},
              'CES_CET_norm': {'CES1': {'f':'CES'}, 'CET1': {'f':'CET_norm'}},
              'CESCET_norm': {'CES1': {'f':'CES_norm'}, 'CET1': {'f': 'CET_norm'}},
              'FunkyTree': {'FunkyTree_CET': {'f':'CET_norm'}, 'FunkyTree_CES': {'f':'CES_norm'}},
              'ExtremeFunk': {'ExtremeF1': {'f': 'CES_norm'}, 'ExtremeF2': {'f':'CET_norm'}},
              'FunnyName': {'FunnyNameInp': {'f':'CES_norm'}, 'FunnyNameOut': {'f':'CET_norm'}}}
Trees = {k: nestingTree.aggTree_from_data(data_str, read_trees = read_trees[k], name = k)(namespace=FunnyName_NS) for k in read_trees} # init b